In [1]:
import pandas as pd
data=pd.read_csv('Suicide_Detection.csv');
data.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [2]:
data = data.drop(data.columns[0], axis=1)

In [3]:
data['class']=data['class'].replace({'suicide': 'depressed', 'non-suicide': 'not depressed'})

In [4]:
data['class'].value_counts()

depressed        116037
not depressed    116037
Name: class, dtype: int64

In [5]:
data=data.iloc[:10000,:]
data.shape
data['class'].value_counts()

not depressed    5085
depressed        4915
Name: class, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
training ,test = train_test_split(data, test_size=0.3, random_state=42)



In [7]:
training['text']
test['class'].value_counts()

depressed        1513
not depressed    1487
Name: class, dtype: int64

In [8]:
X_train=training['text'].values
y_train=training['class'].values
X_test=test['text'].values
y_test=test['class'].values


# Data preparation

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer=TfidfVectorizer()
X_train_final=vectorizer.fit_transform(X_train)
X_test_final=vectorizer.transform(X_test)
#y_train_final=vectorizer.fit_transform(y_train)
#y_test_final=vectorizer.transform(y_test)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier

## SVM

In [11]:
svc=SVC(kernel='linear')
svc.fit(X_train_final,y_train)

SVC(kernel='linear')

## Logistic Regression

In [12]:
lr=LogisticRegression()
lr.fit(X_train_final,y_train)
lr.predict(X_train_final[0])

array(['not depressed'], dtype=object)

## Decision Tree Classifier

In [13]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train_final,y_train)
dtc.predict(X_train_final[0])

array(['not depressed'], dtype=object)

## Random Forest

In [14]:
rfc=RandomForestClassifier()
rfc.fit(X_train_final,y_train)
rfc.predict(X_train_final[0])

array(['not depressed'], dtype=object)

## Ada Boost

In [15]:
abc=AdaBoostClassifier()
abc.fit(X_train_final,y_train)
abc.predict(X_train_final[0])

array(['not depressed'], dtype=object)

## Gradient Boosting

In [16]:
gbc=GradientBoostingClassifier()
gbc.fit(X_train_final,y_train)
gbc.predict(X_train_final[0])

array(['not depressed'], dtype=object)

## KNN

In [17]:
knn=KNeighborsClassifier()
knn.fit(X_train_final,y_train)
knn.predict(X_train_final[0])

/Users/ajaykumarbhardwaj/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array(['not depressed'], dtype=object)

## XGB

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_xgb= le.fit_transform(y_train)
y_test_xgb= le.fit_transform(y_test)
xgb=XGBClassifier()
xgb.fit(X_train_final,y_train_xgb)
xgb.predict(X_train_final[0])

array([1])

# Evaluation


In [19]:
#Mean accuracy
from sklearn.metrics import f1_score,accuracy_score
print(svc.score(X_test_final,y_test))
print(lr.score(X_test_final,y_test))
print(dtc.score(X_test_final,y_test))
print(abc.score(X_test_final,y_test))
print(rfc.score(X_test_final,y_test))
print(xgb.score(X_test_final,y_test_xgb))
print(knn.score(X_test_final,y_test))
print(gbc.score(X_test_final,y_test))

0.919
0.909
0.8156666666666667
0.8816666666666667
0.865
0.899
0.832
0.8903333333333333


/Users/ajaykumarbhardwaj/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [20]:
#F1 scores
from sklearn.metrics import f1_score
f1_score(y_test,lr.predict(X_test_final),average=None)

array([0.90730051, 0.9106383 ])

In [33]:
t=['i am the happiest person']
nt=vectorizer.transform(t)
lr.predict(nt)

array(['not depressed'], dtype=object)

# Hyperparameter Tuning

In [22]:
from sklearn.model_selection import GridSearchCV
# Define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'penalty': ['l1', 'l2'],
              'solver': ['liblinear', 'lbfgs', 'sag'],
              'class_weight': [None, 'balanced']}
# Create a grid search object
grid_search = GridSearchCV(lr, param_grid, cv=5)

# Fit the grid search object to the training data
grid_search.fit(X_train_final, y_train)

# Print the best combination of parameters found by the grid search
print(grid_search.best_params_)

# Use the best combination of parameters to make predictions on the test set
y_pred = grid_search.predict(X_test_final)


/Users/ajaykumarbhardwaj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ajaykumarbhardwaj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

{'C': 10, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}


In [23]:
finalmodel=LogisticRegression(C=10.0, class_weight= None, penalty= 'l2', solver= 'sag')
finalmodel.fit(X_train_final,y_train)
finalmodel.predict(X_test_final)

array(['depressed', 'depressed', 'depressed', ..., 'not depressed',
       'depressed', 'depressed'], dtype=object)

In [24]:
print(finalmodel.score(X_test_final,y_test))

0.9193333333333333


# Saving the model

In [25]:
import pickle

# Save the model to a file
with open('depression_detection_model.pkl', 'wb') as file:
    pickle.dump(finalmodel, file)



In [34]:
loaded_model.predict(nt)

array(['not depressed'], dtype=object)